In [34]:
!pip install fastapi uvicorn pyngrok python-multipart \
             openai-whisper ffmpeg numpy \
             jinja2 aiofiles


In [35]:
!apt update && apt install -y ffmpeg


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
100 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy In

In [36]:
from pyngrok import ngrok
ngrok.set_auth_token("2xfttaPpiNGf2JQPZmxafQBjRcl_3MxVzmsipcCXzJLimLvca")


In [37]:
%%writefile app.py
from fastapi import FastAPI, Request, File, UploadFile
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from fastapi.middleware.cors import CORSMiddleware
import whisper
import tempfile
import os
import uvicorn

app = FastAPI()

# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 정적 파일 및 템플릿 설정
app.mount("/static", StaticFiles(directory="www"), name="static")
templates = Jinja2Templates(directory="www")

# Whisper 모델 로드
print("🎙 Whisper 모델 로딩 중...")
model = whisper.load_model("base")
print("✅ Whisper 모델 로드 완료")

# 인덱스 페이지
@app.get("/", response_class=HTMLResponse)
async def index(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

# 트랜스크립션 API
@app.post("/transcribe")
async def transcribe(file: UploadFile = File(...)):
    with tempfile.NamedTemporaryFile(delete=False, suffix='.webm') as temp_file:
        audio_data = await file.read()
        temp_file.write(audio_data)
        temp_file_path = temp_file.name

    try:
        result = model.transcribe(temp_file_path)
        os.unlink(temp_file_path)
        return JSONResponse(content={"text": result["text"]})
    except Exception as e:
        try:
            os.unlink(temp_file_path)
        except:
            pass
        print(f"❌ 오류 발생: {e}")
        return JSONResponse(content={"error": str(e)}, status_code=500)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=3000)


Overwriting app.py


In [38]:
import os

dirs = ["www"]
for d in dirs:
    os.makedirs(d, exist_ok=True)
    print(f"✅ 디렉토리 생성됨: {d}")


✅ 디렉토리 생성됨: www


In [39]:
%%writefile www/index.html
<!DOCTYPE html>
<html lang="ko">
<head>
  <meta charset="UTF-8">
  <title>Whisper 음성 인식 데모</title>
  <link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Roboto:300,400,500,700&display=swap"/>
  <link rel="stylesheet" href="https://fonts.googleapis.com/icon?family=Material+Icons"/>
  <script src="https://unpkg.com/react@18/umd/react.development.js"></script>
  <script src="https://unpkg.com/react-dom@18/umd/react-dom.development.js"></script>
  <script src="https://unpkg.com/@mui/material@5.14.5/umd/material-ui.development.js"></script>
  <script src="https://unpkg.com/@babel/standalone/babel.min.js"></script>
  <style>
    body { margin: 0; font-family: 'Roboto', sans-serif; }
    .container { max-width: 800px; margin: 0 auto; padding: 20px; }
    .header { background-color: #1976d2; color: white; padding: 16px; margin-bottom: 20px; }
    .chat-bubble { background-color: #f5f5f5; padding: 16px; margin-bottom: 8px; border-radius: 4px; }
    .transcription-container { max-height: 300px; overflow-y: auto; margin-top: 16px; }
    .button-container { display: flex; gap: 8px; margin-top: 16px; }
    .url-input { width: 100%; margin-bottom: 16px; }
  </style>
</head>
<body>
  <div id="root"></div>

  <script type="text/babel">
    const { useState, useRef, useEffect } = React;
    const { AppBar, Toolbar, Typography, Button, Container, Box, TextField } = MaterialUI;

    const App = () => {
      const [isRecording, setIsRecording] = useState(false);
      const [transcriptions, setTranscriptions] = useState([]);
      const [apiUrl, setApiUrl] = useState('');
      const mediaRecorderRef = useRef(null);
      const audioChunksRef = useRef([]);

      useEffect(() => {
        const protocol = window.location.protocol;
        const hostname = window.location.hostname;
        const port = window.location.port;
        const baseUrl = `${protocol}//${hostname}${port ? ':' + port : ''}`;
        setApiUrl(`${baseUrl}/transcribe`);
      }, []);

      const handleApiUrlChange = (event) => {
        setApiUrl(event.target.value);
      };

      const handleStartRecording = async () => {
        try {
          const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
          mediaRecorderRef.current = new MediaRecorder(stream);
          mediaRecorderRef.current.ondataavailable = (event) => {
            audioChunksRef.current.push(event.data);
          };
          mediaRecorderRef.current.onstop = () => {
            sendAudioData();
          };
          audioChunksRef.current = [];
          mediaRecorderRef.current.start();
          setIsRecording(true);
        } catch (error) {
          console.error('Error accessing microphone:', error);
          alert('마이크 접근에 실패했습니다.');
        }
      };

      const handleStopRecording = () => {
        if (mediaRecorderRef.current && isRecording) {
          mediaRecorderRef.current.stop();
          setIsRecording(false);
        }
      };

      const sendAudioData = async () => {
        const audioBlob = new Blob(audioChunksRef.current, { type: 'audio/webm' });
        const formData = new FormData();
        formData.append('file', audioBlob, 'recording.webm');

        try {
          const response = await fetch(apiUrl, {
            method: 'POST',
            body: formData,
          });
          const data = await response.json();
          if (data.text) {
            setTranscriptions((prev) => [...prev, data.text]);
          } else if (data.error) {
            setTranscriptions((prev) => [...prev, `Error: ${data.error}`]);
          }
        } catch (error) {
          console.error('Error sending audio data:', error);
          setTranscriptions((prev) => [...prev, `Error: ${error.message}`]);
        }
      };

      return (
        <Container className="container">
          <div className="header">
            <Typography variant="h6">🎙 Whisper 음성 인식 데모</Typography>
          </div>
          <Box>
            <TextField
              label="API URL"
              variant="outlined"
              fullWidth
              value={apiUrl}
              onChange={handleApiUrlChange}
              className="url-input"
            />
            <div className="button-container">
              <Button
                variant="contained"
                color="primary"
                onClick={handleStartRecording}
                disabled={isRecording}
              >
                녹음 시작
              </Button>
              <Button
                variant="contained"
                color="secondary"
                onClick={handleStopRecording}
                disabled={!isRecording}
              >
                녹음 종료
              </Button>
            </div>
          </Box>
          <div className="transcription-container">
            {transcriptions.map((text, index) => (
              <div key={index} className="chat-bubble">
                {text}
              </div>
            ))}
          </div>
        </Container>
      );
    };

    ReactDOM.render(<App />, document.getElementById('root'));
  </script>
</body>
</html>


Overwriting www/index.html


In [40]:
%%writefile run_server.py
from pyngrok import ngrok
import subprocess
import threading
import time

def run():
    subprocess.run(["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "3000"])

thread = threading.Thread(target=run)
thread.daemon = True
thread.start()

time.sleep(3)

public_url = ngrok.connect(3000).public_url
print("🔗 접속 주소:", public_url)

# 서버를 계속 실행
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("⛔ 서버 종료")
    ngrok.kill()


Overwriting run_server.py


In [41]:
!python run_server.py


🎙 Whisper 모델 로딩 중...
🔗 접속 주소: https://823c-34-106-193-135.ngrok-free.app
✅ Whisper 모델 로드 완료
INFO:     Started server process [23527]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)
INFO:     27.35.110.203:0 - "GET / HTTP/1.1" 200 OK
INFO:     27.35.110.203:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
INFO:     27.35.110.203:0 - "POST /transcribe HTTP/1.1" 200 OK
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
INFO:     27.35.110.203:0 - "POST /transcribe HTTP/1.1" 200 OK
Traceback (most recent call last):
  File "/content/run_server.py", li